In [ ]:
import os
import numpy as np
import logging

from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig
from asgardpy.stats.stats import check_model_preference_aic, check_model_preference_lrt

In [ ]:
# os.environ['GAMMAPY_DATA']

In [ ]:
log = logging.getLogger("Check Preferred Spectral Model")

In [ ]:
config_file = "../src/asgardpy/tests/config_test_gadf.yaml"

# Get config objects for different spectral models

In [ ]:
config_pl = config.read(config_file)
config_bpl = config.read(config_file)
config_sbpl = config.read(config_file)
config_ecpl = config.read(config_file)
config_ecpl2 = config.read(config_file)
config_secpl = config.read(config_file)
config_lp = config.read(config_file)
config_eclp = config.read(config_file)

all_configs = [config_pl, config_bpl, config_sbpl, config_ecpl, config_ecpl2, config_secpl, config_lp, config_eclp]

for config in all_configs:
    config.dataset3d.instruments[0].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}hess-dl3-dr1/"
    config.dataset3d.instruments[0].dataset_info.background.exclusion.exclusion_file = (
        f"{os.environ['GAMMAPY_DATA']}joint-crab/exclusion/exclusion_mask_crab.fits.gz"
    )
    config.dataset1d.instruments[0].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}magic/rad_max/data/"

In [ ]:
config_pl.target.models_file = "../src/asgardpy/config/model_templates/model_template_pl.yaml"
config_bpl.target.models_file = "../src/asgardpy/config/model_templates/model_template_bpl.yaml"
config_sbpl.target.models_file = "../src/asgardpy/config/model_templates/model_template_sbpl.yaml"
config_ecpl.target.models_file = "../src/asgardpy/config/model_templates/model_template_ecpl.yaml"
config_ecpl2.target.models_file = "../src/asgardpy/config/model_templates/model_template_ecpl2.yaml"
config_secpl.target.models_file = "../src/asgardpy/config/model_templates/model_template_secpl.yaml"
config_lp.target.models_file = "../src/asgardpy/config/model_templates/model_template_lp.yaml"
config_eclp.target.models_file = "../src/asgardpy/config/model_templates/model_template_eclp.yaml"

In [ ]:
print(config_pl.target.models_file)
print(config_bpl.target.models_file)
print(config_sbpl.target.models_file)
print(config_ecpl.target.models_file)
print(config_ecpl2.target.models_file)
print(config_secpl.target.models_file)
print(config_lp.target.models_file)
print(config_eclp.target.models_file)

In [ ]:
analysis_pl = AsgardpyAnalysis(config_pl)
analysis_bpl = AsgardpyAnalysis(config_bpl)
analysis_sbpl = AsgardpyAnalysis(config_sbpl)
analysis_ecpl = AsgardpyAnalysis(config_ecpl)
analysis_ecpl2 = AsgardpyAnalysis(config_ecpl2)
analysis_secpl = AsgardpyAnalysis(config_secpl)
analysis_lp = AsgardpyAnalysis(config_lp)
analysis_eclp = AsgardpyAnalysis(config_eclp)

all_analyses = [analysis_pl, analysis_bpl, analysis_sbpl, analysis_ecpl, analysis_ecpl2, analysis_secpl, analysis_lp, analysis_eclp]

# Check the spectral type, redshift (common) and each spectral parameter

In [ ]:
for a in all_analyses:
    s = a.config.target.components[0].spectral
    print(s.type, s.ebl_abs.redshift)
    for p in s.parameters:
        print(p)

# Run Analysis steps

In [ ]:
%%time
for a in all_analyses:
    a.run()

In [ ]:
for a in all_analyses:
    s = a.config.target.components[0].spectral
    print(s.type)
    print(a.fit_result)

In [ ]:
%%time
p_pl_bpl, g_pl_bpl, ndof_pl_bpl = check_model_preference_lrt(
    analysis_pl.instrument_spectral_info["best_fit_stat"], 
    analysis_bpl.instrument_spectral_info["best_fit_stat"], 
    analysis_pl.instrument_spectral_info["DoF"], 
    analysis_bpl.instrument_spectral_info["DoF"]
)
p_pl_sbpl, g_pl_sbpl, ndof_pl_sbpl = check_model_preference_lrt(
    analysis_pl.instrument_spectral_info["best_fit_stat"], 
    analysis_sbpl.instrument_spectral_info["best_fit_stat"], 
    analysis_pl.instrument_spectral_info["DoF"], 
    analysis_sbpl.instrument_spectral_info["DoF"]
)

p_pl_ecpl, g_pl_ecpl, ndof_pl_ecpl = check_model_preference_lrt(
    analysis_pl.instrument_spectral_info["best_fit_stat"], 
    analysis_ecpl.instrument_spectral_info["best_fit_stat"], 
    analysis_pl.instrument_spectral_info["DoF"], 
    analysis_ecpl.instrument_spectral_info["DoF"]
)

p_pl_ecpl2, g_pl_ecpl2, ndof_pl_ecpl2 = check_model_preference_lrt(
    analysis_pl.instrument_spectral_info["best_fit_stat"], 
    analysis_ecpl2.instrument_spectral_info["best_fit_stat"], 
    analysis_pl.instrument_spectral_info["DoF"], 
    analysis_ecpl2.instrument_spectral_info["DoF"], 
)

p_pl_secpl, g_pl_secpl, ndof_pl_secpl = check_model_preference_lrt(
    analysis_pl.instrument_spectral_info["best_fit_stat"], 
    analysis_secpl.instrument_spectral_info["best_fit_stat"], 
    analysis_pl.instrument_spectral_info["DoF"], 
    analysis_secpl.instrument_spectral_info["DoF"], 
)

p_pl_lp, g_pl_lp, ndof_pl_lp = check_model_preference_lrt(
    analysis_pl.instrument_spectral_info["best_fit_stat"], 
    analysis_lp.instrument_spectral_info["best_fit_stat"], 
    analysis_pl.instrument_spectral_info["DoF"], 
    analysis_lp.instrument_spectral_info["DoF"]
)

p_pl_eclp, g_pl_eclp, ndof_pl_eclp = check_model_preference_lrt(
    analysis_pl.instrument_spectral_info["best_fit_stat"], 
    analysis_eclp.instrument_spectral_info["best_fit_stat"], 
    analysis_pl.instrument_spectral_info["DoF"], 
    analysis_eclp.instrument_spectral_info["DoF"]
)

In [ ]:
print(f"Chi2 of goodness of fit for PL: {analysis_pl.instrument_spectral_info['best_fit_stat']:.3f}/{analysis_pl.instrument_spectral_info['DoF']}")
print(f"Chi2 of goodness of fit for BPL: {analysis_bpl.instrument_spectral_info['best_fit_stat']:.3f}/{analysis_bpl.instrument_spectral_info['DoF']}")
print(f"Chi2 of goodness of fit for SBPL: {analysis_sbpl.instrument_spectral_info['best_fit_stat']:.3f}/{analysis_sbpl.instrument_spectral_info['DoF']}")
print(f"Chi2 of goodness of fit for ECPL: {analysis_ecpl.instrument_spectral_info['best_fit_stat']:.3f}/{analysis_ecpl.instrument_spectral_info['DoF']}")
print(f"Chi2 of goodness of fit for ECPL2: {analysis_ecpl2.instrument_spectral_info['best_fit_stat']:.3f}/{analysis_ecpl2.instrument_spectral_info['DoF']}")
print(f"Chi2 of goodness of fit for SECPL: {analysis_secpl.instrument_spectral_info['best_fit_stat']:.3f}/{analysis_secpl.instrument_spectral_info['DoF']}")
print(f"Chi2 of goodness of fit for LP: {analysis_lp.instrument_spectral_info['best_fit_stat']:.3f}/{analysis_lp.instrument_spectral_info['DoF']}")
print(f"Chi2 of goodness of fit for ECLP: {analysis_eclp.instrument_spectral_info['best_fit_stat']:.3f}/{analysis_eclp.instrument_spectral_info['DoF']}")

print(f"p-value of goodness of fit for PL: {analysis_pl.instrument_spectral_info['fit_pval']:.3e}")
print(f"p-value of goodness of fit for BPL: {analysis_bpl.instrument_spectral_info['fit_pval']:.3e}")
print(f"p-value of goodness of fit for SBPL: {analysis_sbpl.instrument_spectral_info['fit_pval']:.3e}")
print(f"p-value of goodness of fit for ECPL: {analysis_ecpl.instrument_spectral_info['fit_pval']:.3e}")
print(f"p-value of goodness of fit for ECPL2: {analysis_ecpl2.instrument_spectral_info['fit_pval']:.3e}")
print(f"p-value of goodness of fit for SECPL: {analysis_secpl.instrument_spectral_info['fit_pval']:.3e}")
print(f"p-value of goodness of fit for LP: {analysis_lp.instrument_spectral_info['fit_pval']:.3e}")
print(f"p-value of goodness of fit for ECLP: {analysis_eclp.instrument_spectral_info['fit_pval']:.3e}")

In [ ]:
print(f"Preference of BPL over PL is {g_pl_bpl:.3f} sigmas")
print(f"Preference of SBPL over PL is {g_pl_sbpl:.3f} sigmas")
print(f"Preference of ECPL over PL is {g_pl_ecpl:.3f} sigmas")
print(f"Preference of ECPL2 over PL is {g_pl_ecpl2:.3f} sigmas")
print(f"Preference of SECPL over PL is {g_pl_secpl:.3f} sigmas")
print(f"Preference of LP over PL is {g_pl_lp:.3f} sigmas")
print(f"Preference of ECLP over PL is {g_pl_eclp:.3f} sigmas")

print(f"p-vaue of BPL over PL is {p_pl_bpl:.3e}")
print(f"p-vaue of SBPL over PL is {p_pl_sbpl:.3e}")
print(f"p-vaue of ECPL over PL is {p_pl_ecpl:.3e}")
print(f"p-vaue of ECPL2 over PL is {p_pl_ecpl2:.3e}")
print(f"p-vaue of SECPL over PL is {p_pl_secpl:.3e}")
print(f"p-vaue of LP over PL is {p_pl_lp:.3e}")
print(f"p-vaue of LP over ECPL is {p_pl_eclp:.3e}")

In [ ]:
list_stat = np.array([
    analysis_pl.instrument_spectral_info["best_fit_stat"],
    analysis_bpl.instrument_spectral_info["best_fit_stat"],
    analysis_sbpl.instrument_spectral_info["best_fit_stat"],
    analysis_ecpl.instrument_spectral_info["best_fit_stat"],
    analysis_ecpl2.instrument_spectral_info["best_fit_stat"],
    analysis_secpl.instrument_spectral_info["best_fit_stat"],
    analysis_lp.instrument_spectral_info["best_fit_stat"],
    analysis_eclp.instrument_spectral_info["best_fit_stat"],
])

list_dof = np.array([
    analysis_pl.instrument_spectral_info['DoF'],
    analysis_bpl.instrument_spectral_info['DoF'],
    analysis_sbpl.instrument_spectral_info['DoF'],
    analysis_ecpl.instrument_spectral_info['DoF'],
    analysis_ecpl2.instrument_spectral_info['DoF'],
    analysis_secpl.instrument_spectral_info['DoF'],
    analysis_lp.instrument_spectral_info['DoF'],
    analysis_eclp.instrument_spectral_info['DoF'],
])

In [ ]:
list_rel_p = check_model_preference_aic(list_stat, list_dof)
print(f"Relative likelihood for PL: {list_rel_p[0]}")
print(f"Relative likelihood for BPL: {list_rel_p[1]}")
print(f"Relative likelihood for SBPL: {list_rel_p[2]}")
print(f"Relative likelihood for ECPL: {list_rel_p[3]}")
print(f"Relative likelihood for ECPL2: {list_rel_p[4]}")
print(f"Relative likelihood for SECPL: {list_rel_p[5]}")
print(f"Relative likelihood for LP: {list_rel_p[6]}")
print(f"Relative likelihood for ECLP: {list_rel_p[7]}")

# Significantly preferred model is when relative likelihood value is > 0.95